In [1]:
import os
import pandas as pd
import tensorflow as tf

from cryptovision.tools import image_directory_to_pandas, split_image_dataframe, tf_dataset_from_pandas

2025-01-28 13:27:09.036 | INFO     | cryptovision.config:<module>:15 - PROJ_ROOT path is: /Users/leonardo/Documents/Projects/cryptovision


In [2]:
SEED = 42

min_samples = 100
test_size = 0.15
val_size = 0.15
batch_size = 128
image_dims = (128, 128)

epochs = 10

In [10]:
df = image_directory_to_pandas(
    '/Volumes/T7_shield/CryptoVision/Data/Images/Datasets/v2.0.0/images'
)

counts = df['species'].value_counts()
df = df[df['species'].isin(counts[counts >= min_samples].index)]

train_df, val_df, test_df = split_image_dataframe(
    df, 
    test_size=test_size, 
    val_size=val_size, 
    stratify_by='folder_label',
    random_state=SEED
)

names = {
    'family': sorted(df['family'].unique()),
    'genus': sorted(df['genus'].unique()),
    'species': sorted(df['species'].unique()),
}

train_ds, _, _, _ = tf_dataset_from_pandas(
    train_df, 
    batch_size=batch_size, 
    image_size=image_dims,
)

val_ds, _, _, _ = tf_dataset_from_pandas(
    val_df, 
    batch_size=batch_size, 
    image_size=image_dims,
)

test_ds, _, _, _ = tf_dataset_from_pandas(
    test_df, 
    batch_size=batch_size, 
    image_size=image_dims,
)

train_ds = train_ds.cache().shuffle(buffer_size=1000, seed=SEED).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [9]:
df[df['species'] == 'Apogon binotatus'].shape

(233, 5)

In [11]:
df[df['species'] == 'Apogon binotatus'].shape

(184, 5)

In [18]:
sorted(df['folder_label'].unique().tolist())

['Apogonidae_Apogon_binotatus',
 'Apogonidae_Apogon_maculatus',
 'Apogonidae_Cheilodipterus_quinquelineatus',
 'Apogonidae_Ostorhinchus_angustatus',
 'Apogonidae_Ostorhinchus_compressus',
 'Apogonidae_Ostorhinchus_cyanosoma',
 'Apogonidae_Ostorhinchus_nigrofasciatus',
 'Apogonidae_Pristiapogon_exostigma',
 'Apogonidae_Pristiapogon_kallopterus',
 'Apogonidae_Taeniamia_fucata',
 'Blenniidae_Cirripectes_variolosus',
 'Blenniidae_Crossosalarias_macrospilus',
 'Blenniidae_Ecsenius_bicolor',
 'Blenniidae_Ecsenius_stictus',
 'Blenniidae_Glyptoparus_delicatulus',
 'Blenniidae_Plagiotremus_rhinorhynchus',
 'Blenniidae_Salarias_alboguttatus',
 'Chaenopsidae_Acanthemblemaria_aspera',
 'Chaenopsidae_Acanthemblemaria_spinosa',
 'Chaetodontidae_Chaetodon_citrinellus',
 'Chaetodontidae_Chaetodon_quadrimaculatus',
 'Cirrhitidae_Cirrhitichthys_oxycephalus',
 'Cirrhitidae_Neocirrhites_armatus',
 'Cirrhitidae_Paracirrhites_arcatus',
 'Gobiidae_Amblygobius_phalaena',
 'Gobiidae_Coryphopterus_personatus',


In [12]:
from cryptovision.models import basic_multioutput, keras_apps

pretrain = keras_apps.ResNet50V2(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
preprocess = keras_apps.resnet_v2.preprocess_input

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomTranslation(0.1, 0.1),
        tf.keras.layers.RandomContrast(0.2),
        tf.keras.layers.RandomBrightness(0.2),
    ]
)

model = basic_multioutput(
    pretrain=pretrain,
    preprocess=preprocess,
    input_shape=(128, 128, 3),
    outputs_size=[len(names['family']), len(names['genus']), len(names['species'])],
    dropout_rate=0.3,
    augmentation=data_augmentation,
)

model.summary()

Model: "MultiOutputBasic"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_3 (Cast)       │ (None, 128, 128,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 128, 128,  │          0 │ cast_3[0][0]      │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ true_divide_1       │ (None, 128, 128,  │          0 │ sequential_1[0][… │
│ (TrueDivide)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_1          │ (None, 128, 128,  │          0 │ true_divide_1[0]… │
│ (Subtract)          │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50v2          │ (None, 4, 4,      │ 23,564,800 │ subtract_1[0][0]  │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GlobMaxPool2D       │ (None, 2048)      │          0 │ resnet50v2[0][0]  │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer        │ (None, 2048)      │  4,196,352 │ GlobMaxPool2D[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2048)      │      8,192 │ shared_layer[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 2048)      │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 2048)      │          0 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ family (Dense)      │ (None, 18)        │     36,882 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ genus (Dense)       │ (None, 51)        │    104,499 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ species (Dense)     │ (None, 86)        │    176,214 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 28,086,939 (107.14 MB)

 Trainable params: 4,518,043 (17.23 MB)

 Non-trainable params: 23,568,896 (89.91 MB)

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss= {
        "family": "categorical_focal_crossentropy",
        "genus": "categorical_focal_crossentropy",
        "species": "categorical_focal_crossentropy",
    },
    metrics={
        "family": ["accuracy", "AUC", "Precision", "Recall"],
        "genus": ["accuracy", "AUC", "Precision", "Recall"],
        "species": ["accuracy", "AUC", "Precision", "Recall"],
    },
)   

In [14]:
from cryptovision.tools import TQDMProgressBar

history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=[TQDMProgressBar()],
    verbose=0,
)

Epoch 1/10:   0%|          | 0/129 [00:00<?, ?batch/s]2025-01-28 13:49:36.587235: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:397: Filling up shuffle buffer (this may take a while): 32 of 16468
2025-01-28 13:49:46.641051: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:397: Filling up shuffle buffer (this may take a while): 65 of 16468
2025-01-28 13:50:05.898968: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
2025-01-28 13:50:05.898993: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:411: Filling up shuffle buffer (this may take a while): 1 of 1000
2025-01-28 13:50:05.899149: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
Epoch 1/10: 100%|██████████| 129/129 [01:35<00:00,  1.36batch/s, loss=2.1189, family_acc=0.3613, genus_acc=0.2210, species_acc=0.1470]


2025-01-28 13:50:55.930 | INFO     | cryptovision.tools:on_epoch_end:86 - Epoch 1 completed - Loss: 2.1189, Val Family Accuracy: 0.5248, Val Genus Accuracy: 0.3800, Val Species Accuracy: 0.2919


Epoch 2/10: 100%|██████████| 129/129 [00:35<00:00,  3.67batch/s, loss=1.5382, family_acc=0.4836, genus_acc=0.3605, species_acc=0.2922]


2025-01-28 13:51:31.123 | INFO     | cryptovision.tools:on_epoch_end:86 - Epoch 2 completed - Loss: 1.5382, Val Family Accuracy: 0.5653, Val Genus Accuracy: 0.4372, Val Species Accuracy: 0.3519


Epoch 3/10: 100%|██████████| 129/129 [00:34<00:00,  3.69batch/s, loss=1.3757, family_acc=0.5186, genus_acc=0.4077, species_acc=0.3368]


...

2025-01-28 13:54:59.297 | INFO     | cryptovision.tools:on_epoch_end:86 - Epoch 8 completed - Loss: 1.0986, Val Family Accuracy: 0.6095, Val Genus Accuracy: 0.5211, Val Species Accuracy: 0.4523


Epoch 9/10: 100%|██████████| 129/129 [00:35<00:00,  3.59batch/s, loss=1.0831, family_acc=0.5783, genus_acc=0.4902, species_acc=0.4364]


2025-01-28 13:55:35.256 | INFO     | cryptovision.tools:on_epoch_end:86 - Epoch 9 completed - Loss: 1.0831, Val Family Accuracy: 0.6135, Val Genus Accuracy: 0.5191, Val Species Accuracy: 0.4531


Epoch 10/10: 100%|██████████| 129/129 [00:35<00:00,  3.64batch/s, loss=1.0561, family_acc=0.5862, genus_acc=0.4991, species_acc=0.4443]

2025-01-28 13:56:10.710 | INFO     | cryptovision.tools:on_epoch_end:86 - Epoch 10 completed - Loss: 1.0561, Val Family Accuracy: 0.6109, Val Genus Accuracy: 0.5234, Val Species Accuracy: 0.4568


In [7]:
results = model.evaluate(test_ds, return_dict=True)

print(f"""
            -- Report for HACPL-MO Classifier --\n
Family Accuracy: {results['family_accuracy']:.3f} / Loss: {results['family_loss']:.3f}\n
Genus Accuracy: {results['genus_accuracy']:.3f} / Loss: {results['genus_loss']:.3f}\n
Species Accuracy: {results['species_accuracy']:.3f} / Loss: {results['species_loss']:.3f}

""")

2025-01-28 13:34:22.163999: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:17: Filling up shuffle buffer (this may take a while): 26 of 3537
2025-01-28 13:34:22.761818: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 194ms/step - family_AUC: 0.9566 - family_Precision: 0.8290 - family_Recall: 0.4889 - family_accuracy: 0.6394 - family_loss: 0.1970 - genus_AUC: 0.9495 - genus_Precision: 0.8024 - genus_Recall: 0.3747 - genus_accuracy: 0.5455 - genus_loss: 0.3392 - loss: 0.9730 - species_AUC: 0.9435 - species_Precision: 0.8190 - species_Recall: 0.3032 - species_accuracy: 0.4738 - species_loss: 0.4392

            -- Report for HACPL-MO Classifier --

Family Accuracy: 0.642 / Loss: 0.197

Genus Accuracy: 0.547 / Loss: 0.340

Species Accuracy: 0.469 / Loss: 0.438




In [15]:
results = model.evaluate(test_ds, return_dict=True)

print(f"""
            -- Report for HACPL-MO Classifier --\n
Family Accuracy: {results['family_accuracy']:.3f} / Loss: {results['family_loss']:.3f}\n
Genus Accuracy: {results['genus_accuracy']:.3f} / Loss: {results['genus_loss']:.3f}\n
Species Accuracy: {results['species_accuracy']:.3f} / Loss: {results['species_loss']:.3f}

""")

28/28 ━━━━━━━━━━━━━━━━━━━━ 14s 187ms/step - family_AUC: 0.9549 - family_Precision: 0.8221 - family_Recall: 0.4662 - family_accuracy: 0.6268 - family_loss: 0.2013 - genus_AUC: 0.9491 - genus_Precision: 0.8050 - genus_Recall: 0.3582 - genus_accuracy: 0.5228 - genus_loss: 0.3481 - loss: 0.9893 - species_AUC: 0.9467 - species_Precision: 0.8104 - species_Recall: 0.2810 - species_accuracy: 0.4633 - species_loss: 0.4397

            -- Report for HACPL-MO Classifier --

Family Accuracy: 0.634 / Loss: 0.196

Genus Accuracy: 0.538 / Loss: 0.335

Species Accuracy: 0.472 / Loss: 0.430




In [21]:
species_df = metrics_summary['Species']['metrics']

In [22]:
species_df.head()

,Precision,Recall,F1-Score,Support
Class,,,,
Acanthemblemaria aspera,0.670000,0.858974,0.752809,78
Acanthemblemaria spinosa,0.463415,0.513514,0.487179,37
Amblygobius phalaena,0.166667,0.193548,0.179104,31
Apogon binotatus,0.741935,0.821429,0.779661,28
Apogon maculatus,0.578947,0.589286,0.584071,56


In [8]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def evaluate_per_class_metrics(test_ds, model, family_labels, genus_labels, species_labels):
    """
    Evaluate accuracy, precision, recall, and support for each class across Family, Genus, and Species predictions.

    Parameters:
        test_ds (tf.data.Dataset): Test dataset
        model (tf.keras.Model): Trained model
        family_labels, genus_labels, species_labels (list): Unique labels for Family, Genus, and Species

    Returns:
        metrics_summary (dict): Dictionary containing metrics for Family, Genus, and Species
    """
    all_true_families = []
    all_pred_families = []
    all_true_genus = []
    all_pred_genus = []
    all_true_species = []
    all_pred_species = []

    # Collect all predictions and true labels
    for batch in test_ds:
        images, labels = batch
        family_true = tf.argmax(labels['family'], axis=1).numpy()
        genus_true = tf.argmax(labels['genus'], axis=1).numpy()
        species_true = tf.argmax(labels['species'], axis=1).numpy()

        preds = model.predict(images, verbose=0)
        family_pred = tf.argmax(preds[0], axis=1).numpy()
        genus_pred = tf.argmax(preds[1], axis=1).numpy()
        species_pred = tf.argmax(preds[2], axis=1).numpy()

        all_true_families.extend(family_true)
        all_pred_families.extend(family_pred)
        all_true_genus.extend(genus_true)
        all_pred_genus.extend(genus_pred)
        all_true_species.extend(species_true)
        all_pred_species.extend(species_pred)

    # Helper function to calculate metrics per class
    def calculate_metrics_per_class(y_true, y_pred, labels):
        precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, labels=range(len(labels)))
        accuracy = accuracy_score(y_true, y_pred)
        metrics = pd.DataFrame({
            'Class': labels,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'Support': support
        }).set_index('Class')
        return metrics, accuracy

    # Calculate metrics for each output
    family_metrics, family_accuracy = calculate_metrics_per_class(all_true_families, all_pred_families, family_labels)
    genus_metrics, genus_accuracy = calculate_metrics_per_class(all_true_genus, all_pred_genus, genus_labels)
    species_metrics, species_accuracy = calculate_metrics_per_class(all_true_species, all_pred_species, species_labels)

    # Display and summarize results
    print("\nFamily-Level Metrics")
    print(f"Accuracy: {family_accuracy:.4f}")
    display(family_metrics)

    print("\nGenus-Level Metrics")
    print(f"Accuracy: {genus_accuracy:.4f}")
    display(genus_metrics)

    print("\nSpecies-Level Metrics")
    print(f"Accuracy: {species_accuracy:.4f}")
    display(species_metrics)

    # Summarize results in a dictionary
    metrics_summary = {
        'Family': {'metrics': family_metrics, 'accuracy': family_accuracy},
        'Genus': {'metrics': genus_metrics, 'accuracy': genus_accuracy},
        'Species': {'metrics': species_metrics, 'accuracy': species_accuracy}
    }

    return metrics_summary

# Example usage
metrics_summary = evaluate_per_class_metrics(
    test_ds, 
    model, 
    names['family'], 
    names['genus'], 
    names['species']
)


Family-Level Metrics
Accuracy: 0.6418


2025-01-28 13:34:39.386695: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,Precision,Recall,F1-Score,Support
Class,,,,
Apogonidae,0.675532,0.679144,0.677333,374
Blenniidae,0.428571,0.406542,0.417266,214
Chaenopsidae,0.758065,0.817391,0.786611,115
Chaetodontidae,0.846154,0.692913,0.761905,127
Cirrhitidae,0.762712,0.375000,0.502793,120
Gobiidae,0.597403,0.789700,0.680222,699
Labridae,0.621711,0.462103,0.530154,409
Monacanthidae,0.535714,0.416667,0.468750,36
Pinguipedidae,0.666667,0.400000,0.500000,30



Genus-Level Metrics
Accuracy: 0.5468


,Precision,Recall,F1-Score,Support
Class,,,,
Acanthemblemaria,0.662252,0.869565,0.751880,115
Amblygobius,0.296296,0.258065,0.275862,31
Apogon,0.736264,0.736264,0.736264,91
Canthigaster,0.453947,0.644860,0.532819,107
Caracanthus,0.760000,0.760000,0.760000,25
Centropyge,0.750000,0.647727,0.695122,88
Cephalopholis,0.423077,0.366667,0.392857,30
Chaetodon,0.752000,0.740157,0.746032,127
Cheilodipterus,0.500000,0.303030,0.377358,33



Species-Level Metrics
Accuracy: 0.4693


,Precision,Recall,F1-Score,Support
Class,,,,
Acanthemblemaria aspera,0.618557,0.769231,0.685714,78
Acanthemblemaria spinosa,0.466667,0.567568,0.512195,37
Amblygobius phalaena,0.254902,0.419355,0.317073,31
Apogon binotatus,0.714286,0.571429,0.634921,35
Apogon maculatus,0.596491,0.607143,0.601770,56
...,...,...,...,...
Sebastapistes fowleri,0.571429,0.833333,0.677966,24
Taeniamia fucata,0.200000,0.161290,0.178571,31
Thalassoma amblycephalum,0.266667,0.133333,0.177778,30


In [16]:
# Example usage
metrics_summary = evaluate_per_class_metrics(
    test_ds, 
    model, 
    names['family'], 
    names['genus'], 
    names['species']
)


Family-Level Metrics
Accuracy: 0.6339


2025-01-28 13:58:39.553182: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,Precision,Recall,F1-Score,Support
Class,,,,
Apogonidae,0.675141,0.651226,0.662968,367
Blenniidae,0.440191,0.429907,0.434988,214
Chaenopsidae,0.762712,0.782609,0.772532,115
Chaetodontidae,0.945652,0.685039,0.794521,127
Cirrhitidae,0.617978,0.458333,0.526316,120
Gobiidae,0.625759,0.736767,0.676741,699
Labridae,0.589947,0.546569,0.567430,408
Monacanthidae,0.608696,0.388889,0.474576,36
Pinguipedidae,0.857143,0.200000,0.324324,30



Genus-Level Metrics
Accuracy: 0.5378


,Precision,Recall,F1-Score,Support
Class,,,,
Acanthemblemaria,0.671429,0.817391,0.737255,115
Amblygobius,0.147059,0.161290,0.153846,31
Apogon,0.746988,0.738095,0.742515,84
Canthigaster,0.432836,0.542056,0.481328,107
Caracanthus,0.548387,0.680000,0.607143,25
Centropyge,0.697368,0.602273,0.646341,88
Cephalopholis,0.500000,0.200000,0.285714,30
Chaetodon,0.867925,0.724409,0.789700,127
Cheilodipterus,0.500000,0.121212,0.195122,33



Species-Level Metrics
Accuracy: 0.4715


,Precision,Recall,F1-Score,Support
Class,,,,
Acanthemblemaria aspera,0.670000,0.858974,0.752809,78
Acanthemblemaria spinosa,0.463415,0.513514,0.487179,37
Amblygobius phalaena,0.166667,0.193548,0.179104,31
Apogon binotatus,0.741935,0.821429,0.779661,28
Apogon maculatus,0.578947,0.589286,0.584071,56
...,...,...,...,...
Sebastapistes fowleri,0.714286,0.833333,0.769231,24
Taeniamia fucata,0.285714,0.129032,0.177778,31
Thalassoma amblycephalum,0.368421,0.233333,0.285714,30


In [1]:
l = [
    #'Apogonidae_Apogon_binotatus',
    #'Apogonidae_Apogon_maculatus',
    #'Apogonidae_Cheilodipterus_quinquelineatus',
    #'Apogonidae_Ostorhinchus_angustatus',
    #'Apogonidae_Ostorhinchus_compressus',
    #'Apogonidae_Ostorhinchus_cyanosoma',
    #'Apogonidae_Ostorhinchus_nigrofasciatus',
    #'Apogonidae_Pristiapogon_exostigma',
    #'Apogonidae_Pristiapogon_kallopterus',
    #'Apogonidae_Taeniamia_fucata',
    #'Blenniidae_Cirripectes_variolosus',
    #'Blenniidae_Crossosalarias_macrospilus',
    #'Blenniidae_Ecsenius_bicolor',
    #'Blenniidae_Ecsenius_stictus',
    #'Blenniidae_Glyptoparus_delicatulus',
    #'Blenniidae_Plagiotremus_rhinorhynchus',
    #'Blenniidae_Salarias_alboguttatus',
    #'Chaenopsidae_Acanthemblemaria_aspera',
    #'Chaenopsidae_Acanthemblemaria_spinosa',
    #'Chaetodontidae_Chaetodon_citrinellus',
    #'Chaetodontidae_Chaetodon_quadrimaculatus',
    #'Cirrhitidae_Cirrhitichthys_oxycephalus',
    #'Cirrhitidae_Neocirrhites_armatus',
    #'Cirrhitidae_Paracirrhites_arcatus',
    #'Gobiidae_Amblygobius_phalaena',
    #'Gobiidae_Coryphopterus_personatus',
    #'Gobiidae_Eviota_atriventris',
    'Gobiidae_Eviota_prasites',
    'Gobiidae_Eviota_sebreei',
    'Gobiidae_Eviota_teresae',
    'Gobiidae_Fusigobius_duospilus',
    'Gobiidae_Fusigobius_neophytus',
    'Gobiidae_Fusigobius_signipinnis',
    'Gobiidae_Gobiodon_quinquestrigatus',
    'Gobiidae_Istigobius_decoratus',
    'Gobiidae_Istigobius_decoratus ',
    'Gobiidae_Istigobius_rigilius',
    'Gobiidae_Koumansetta_rainfordi',
    'Gobiidae_Nemateleotris_magnifica',
    'Labridae_Coris_batuensis',
    'Labridae_Gomphosus_varius',
    'Labridae_Halichoeres_chloropterus',
    'Labridae_Halichoeres_claudia',
    'Labridae_Halichoeres_melanurus',
    'Labridae_Labroides_dimidiatus',
    'Labridae_Pseudocheilinus_hexataenia',
    'Labridae_Thalassoma_amblycephalum',
    'Labridae_Thalassoma_lunare',
    'Monacanthidae_Monacanthus_tuckeri',
    'Pinguipedidae_Parapercis_cylindrica',
    'Pomacanthidae_Centropyge_bispinosa',
    'Pomacanthidae_Centropyge_flavissima',
    'Pomacanthidae_Pomacanthus_maculosus',
    'Pomacentridae_Chromis_atripectoralis',
    'Pomacentridae_Chromis_iomelas',
    'Pomacentridae_Chromis_lepidolepis',
    'Pomacentridae_Chromis_margaritifer',
    'Pomacentridae_Chrysiptera_caesifrons',
    'Pomacentridae_Chrysiptera_rollandi',
    'Pomacentridae_Dascyllus_aruanus',
    'Pomacentridae_Dascyllus_reticulatus',
    'Pomacentridae_Neopomacentrus_azysron',
    'Pomacentridae_Neopomacentrus_cyanomos',
    'Pomacentridae_Pomacentrus_adelus',
    'Pomacentridae_Pomacentrus_amboinensis',
    'Pomacentridae_Pomacentrus_bankanensis',
    'Pomacentridae_Pomacentrus_brachialis',
    'Pomacentridae_Pomacentrus_chrysurus',
    'Pomacentridae_Pomacentrus_lepidogenys',
    'Pomacentridae_Pomacentrus_moluccensis',
    'Pomacentridae_Pomacentrus_nagasakiensis',
    'Pomacentridae_Pomacentrus_pavo',
    'Pseudochromidae_Cypho_purpurascens',
    'Pseudochromidae_Pseudochromis_fuscus',
    'Sciaenidae_Eques_punctatus',
    'Scorpaenidae_Caracanthus_maculatus',
    'Scorpaenidae_Scorpaenodes_parvipinnis',
    'Scorpaenidae_Sebastapistes_fowleri',
    'Serranidae_Cephalopholis_cyanostigma',
    'Syngnathidae_Corythoichthys_conspicillatus',
    'Syngnathidae_Corythoichthys_flavofasciatus',
    'Syngnathidae_Doryrhamphus_excisus',
    'Tetraodontidae_Canthigaster_solandri',
    'Tripterygiidae_Enneapterygius_similis',
    'Tripterygiidae_Enneapterygius_tutuilae',
    'Tripterygiidae_Ucla_xenogrammus'
]

len(l)

67